## Imports

In [32]:
# %cd /home/liamroy/Documents/PHD/repos/RL_audio/notebooks
# %cd <add your path here and comment out the others>

%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_03/LLM_motion/llm_audio_testcase'

In [33]:
# IMPORTS
import os
import shutil
import time
import sys
import random

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.ticker import FormatStrFormatter

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd

from scipy import stats

from termcolor import colored, cprint
# Termcolor guide: https://pypi.org/project/termcolor/

from openpyxl import Workbook
from openpyxl import load_workbook

%matplotlib widget
# %matplotlib inline
# %matplotlib notebook

# Try to use this website to use the explode feature, so we can see internal blocks and space everything out
# https://terbium.io/2017/12/matplotlib-3d/ 

# Use this website to make your GIFs - generally 50 delay per frame is good
# https://ezgif.com/maker

## Initializations

In [34]:
def explode(data):
    shape_arr = np.array(data.shape)
    size = shape_arr[:3]*2 - 1
    exploded = np.zeros(np.concatenate([size, shape_arr[3:]]), dtype=data.dtype)
    exploded[::2, ::2, ::2] = data
    return exploded

def plot_3d_array(vox_array, save_str, title_str, title_size=35,
                  x_label='\n\n100    -    140    -    180\n\nBeat Per Min',
                  y_label='\n\n1 - 2 - 4\n\nBeat Per Loop',
                  z_label='\n\n\nNeg - Neutr - Pos\n\nPitch Bend\n',
                  label_size=35,
                  norm_min=-10, norm_max=10, 
                  scale_label="Action Value", colour_map="RdYlGn", plotter=True,):
    
    ''' Plots a 3D array using voxels. 
        vox_array = your 3D aray as a .npz file
        save_str  = what you want the filename to be saved as
        title_str = the title for your plot
        plotter   = whether you want the plot saved (None) or plotted in this notebook (True)
        norm_min  = what the minimum on scale is 
        norm_max  = what the maximum on scale is
        scale_label = obvious
        colour_map = how the colour is mapped - see link: https://matplotlib.org/stable/tutorials/colors/colormaps.html'''
    
    
    # Create the figure
    fig = plt.figure(figsize=(20, 25)) #14 - 12

    
    # Make it a 3D plot
    ax = fig.add_subplot(projection='3d')

    
    plt.title(f"{title_str}", size=title_size, y=0.95)  #20

    
    # Set title, axes and ticks
    # ax.set_title(f"{title_str}", size=25)

    ax.set_xticks([]) # for minor ticks --> ax.set_xticks([], minor=True)
    ax.set_yticks([])
    ax.set_zticks([])

    ax.set_xlabel(x_label, size=label_size)
    ax.set_ylabel(y_label, size=label_size)
    ax.set_zlabel(z_label, size=label_size)


    # Set no grid and desired view angle
    ax.grid(False)
    ax.view_init(8, 285)


    # Define the colour map
    cmap = plt.get_cmap(colour_map)
    # norm= plt.Normalize(vox_array.min(), vox_array.max())
    norm= plt.Normalize(norm_min, norm_max)


    # Creates an array with a 1 at each location of the array where we want a voxel, then explode
    filled = np.ones_like(vox_array)
    filled = explode(filled)

    # Create a colourmap & explode
    colors = cmap(norm(vox_array), alpha=0.85)
    colors = explode(colors)
        
        
    # Plot the voxels 
    ax.voxels(filled, facecolors=colors, edgecolors='black', shade=False)


    # To get the colour bar (note scatter size is set to zero - workaround since no cbar w/ voxels)
    counter = range(3)
    x,y,z = np.meshgrid(counter, counter, counter)
    scatter_cube = ax.scatter(x,y,z, c=vox_array.flat, cmap=plt.get_cmap(colour_map), s=0, alpha=0.8, linewidths=2, edgecolors="black", vmin=norm_min, vmax=norm_max)
    cbar = fig.colorbar(scatter_cube, shrink=0.45, orientation="vertical", aspect=8, pad=0.2) # Add a color bar
    cbar.minorticks_off()
    cbar.set_label(f'\n{scale_label}', rotation=90, size=35) #15
    cbar.ax.tick_params(labelsize=25)

    # Save the fig
    plt.savefig(save_str, bbox_inches='tight', pad_inches=0.25)

    # Show the fig if plotter is not set to None
    if plotter:
        plt.show()

    
    if plotter == None:
        print(" Plots closed. ")
        matplotlib.pyplot.close()


# Matrix Plottter

* Plottter saves all as .png files
* First lets read in our data and create .npy arrays for our histograms

In [35]:
state_sheet_names = ['stuck', 'accomplished', 'progressing']
attempt_ID = "00"


for sheet_name in state_sheet_names:
    file_path = 'llm_audio_testcase.xlsx'  # Adjust the path if necessary
    data_range = 'D2:F81'
    
    df = pd.read_excel(file_path, sheet_name=sheet_name + "_" + attempt_ID, usecols="D:F", nrows=80)

    df = df.fillna('B')
    
    # Mapping values 'A', 'B', 'C' to indices 0, 1, 2
    mapping = {'A': 0, 'B': 1, 'C': 2}
    
    # Initialize a 3x3x3 array to count occurrences
    single_array = np.zeros((3, 3, 3), dtype=int)
    
    # Iterate through each row in the DataFrame and update the histogram
    for _, row in df.iterrows():
        # Get the index values corresponding to the current row's parameters
        idx1 = mapping[row['P1 BPM']]
        idx2 = mapping[row['P2 BPL']]
        idx3 = mapping[row['P3 PitchBend']]
        
        # Increment the histogram at the corresponding indices
        single_array[idx1, idx2, idx3] += 1
    
    # Save the histogram as a .npy file
    np.save("np_arrays/" + sheet_name + '_histogram.npy', single_array)
    
    # Print or inspect the histogram if needed
    print(single_array)
    
    # Now plot
    
    # Load an array or use a custom array
    # single_array = np.array(
    #     [[[-1, 0, 1], [0, 1, 3], [1, 3, 5]],
    #      [[0, 1, 3], [1, 3, 5], [3, 5, 10]],
    #      [[-1, 0, 1], [0, 1, 3], [1, 3, 5]]])
    
    # Customize your axes labels
    # single_x_label='\n\nLeft     -     Centre     -     Right\n\nBody Tilt'
    # single_y_label='\n\nDown - Neutr - Up\n\nHead Angle'
    # single_z_label='\n\n\nDown  -  Neutr  -  Up\n\nBody Height\n'
    
    
    # Customize your title and save file name
    single_label_size=25
    single_title_str = "LLM-Generated Audio Expression\nSelection Heatmap - State: " + sheet_name
    single_save_str = "plots/llm_" + sheet_name + "_heatmap.png"
    
    plot_3d_array(single_array, single_save_str, single_title_str, title_size=40,
                  # x_label=single_x_label,
                  # y_label=single_y_label,
                  # z_label=single_z_label,
                  label_size=single_label_size,
                  norm_min=0, norm_max=80, 
                  scale_label="Count", colour_map='Greens',
                  plotter=None)

[[[69  0  0]
  [ 2  0  0]
  [ 9  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
 Plots closed. 
[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  2]
  [ 0  0 24]
  [ 0  0 33]]

 [[ 0  0  0]
  [ 0  0 12]
  [ 0  0  9]]]
 Plots closed. 
[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  2 78]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
 Plots closed. 


In [ ]:
# Initializations ( NO SET REQUIRED ) 
param_disc = 3

condition_list = ["sect2X", "sect2O"]

all_list = [*range(3, 27)]


for condition_str in condition_list:

    std_array_st0 = np.zeros((param_disc, param_disc, param_disc))
    std_array_st1 = np.zeros((param_disc, param_disc, param_disc))
    std_array_st2 = np.zeros((param_disc, param_disc, param_disc))
    
    mean_array_st0 = np.zeros((param_disc, param_disc, param_disc))
    mean_array_st1 = np.zeros((param_disc, param_disc, param_disc))
    mean_array_st2 = np.zeros((param_disc, param_disc, param_disc))    
    
    print(f"****************************************************\nCONDITON: {condition_str}\n****************************************************\n")
    
    # Itterate param indecies.
    # Recall: param_0 = Speed of audio loop (BPM)         --> (100 BPM   //  140 BPM  //  180 BPM)
    #         param_1 = Number of beeps per loop (BPL)   --> (1 BPL     //  2 BPM     //  4 BPL) 
    #         param_2 = Amplitude of Pitch Bend           --> (downward  //  neutral  //  upward)

    for param_0_idx in range(0, 3):
        for param_1_idx in range(0, 3):
            for param_2_idx in range(0, 3):

                # Reset varriance list to empty
                st0_var_list = [] 
                st1_var_list = [] 
                st2_var_list = [] 


                for user_ID_idx in all_list:
                    user_ID_load_str = f"{user_ID_idx:02}"
                    load_file = user_ID_load_str + "_" + condition_str + "_final"

                    st0_array_path_str = "user_data/user_" + user_ID_load_str + "/arrays/" + load_file + "_st0.npy"
                    st1_array_path_str = "user_data/user_" + user_ID_load_str + "/arrays/" + load_file + "_st1.npy"
                    st2_array_path_str = "user_data/user_" + user_ID_load_str + "/arrays/" + load_file + "_st2.npy"
                    # print(st0_array_path_str)
                    # print(st1_array_path_str)
                    # print(st2_array_path_str)
                    # print()

                    st0_var_list.append(np.load(st0_array_path_str)[param_0_idx, param_1_idx, param_2_idx])

                    st1_var_list.append(np.load(st1_array_path_str)[param_0_idx, param_1_idx, param_2_idx])

                    st2_var_list.append(np.load(st2_array_path_str)[param_0_idx, param_1_idx, param_2_idx])

                print(f"INDEX: {param_0_idx} {param_1_idx} {param_2_idx}")
                print("st0_var_list:", len(st0_var_list), st0_var_list)
                print("st1_var_list:", len(st1_var_list), st1_var_list)
                print("st2_var_list:", len(st2_var_list), st2_var_list)
                print()

                st0_var_list_std = np.std(st0_var_list)
                st1_var_list_std = np.std(st1_var_list)
                st2_var_list_std = np.std(st2_var_list)
                
                print("st0_var_list_std:", st0_var_list_std)
                print("st1_var_list_std:", st1_var_list_std)
                print("st2_var_list_std:", st2_var_list_std)
                print()
                
                st0_var_list_mean = np.mean(st0_var_list)
                st1_var_list_mean = np.mean(st1_var_list)
                st2_var_list_mean = np.mean(st2_var_list)
                
                print("st0_var_list_mean:", st0_var_list_mean)
                print("st1_var_list_mean:", st1_var_list_mean)
                print("st2_var_list_mean:", st2_var_list_mean)
                print()

                std_array_st0[param_0_idx, param_1_idx, param_2_idx] = st0_var_list_std
                std_array_st1[param_0_idx, param_1_idx, param_2_idx] = st1_var_list_std
                std_array_st2[param_0_idx, param_1_idx, param_2_idx] = st2_var_list_std
                
                mean_array_st0[param_0_idx, param_1_idx, param_2_idx] = st0_var_list_mean
                mean_array_st1[param_0_idx, param_1_idx, param_2_idx] = st1_var_list_mean
                mean_array_st2[param_0_idx, param_1_idx, param_2_idx] = st2_var_list_mean
            
                
    print("std_array_st0:\n", std_array_st0, "\n\n")
    print("std_array_st1:\n", std_array_st1, "\n\n")
    print("std_array_st2:\n", std_array_st2, "\n\n")
    
    print("mean_array_st0:\n", mean_array_st0, "\n\n")
    print("mean_array_st1:\n", mean_array_st1, "\n\n")
    print("mean_array_st2:\n", mean_array_st2, "\n\n")
    
    # Save the arrays:
    np.save("user_data/user_all/arrays/st0_variance_" + condition_str + "_matrix.npy", std_array_st0)    
    np.save("user_data/user_all/arrays/st1_variance_" + condition_str + "_matrix.npy", std_array_st1)    
    np.save("user_data/user_all/arrays/st2_variance_" + condition_str + "_matrix.npy", std_array_st2)    
    
    np.save("user_data/user_all/arrays/st0_mean_" + condition_str + "_matrix.npy", mean_array_st0)    
    np.save("user_data/user_all/arrays/st1_mean_" + condition_str + "_matrix.npy", mean_array_st1)    
    np.save("user_data/user_all/arrays/st2_mean_" + condition_str + "_matrix.npy", mean_array_st2)    

### Now plot all the variance matrices

In [ ]:
condition_list = ["sect2X", "sect2O"]

for condition_str in condition_list:
        
    var_array_st0 = np.load("user_data/user_all/arrays/st0_variance_" + condition_str + "_matrix.npy")
    var_array_st1 = np.load("user_data/user_all/arrays/st1_variance_" + condition_str + "_matrix.npy")
    var_array_st2 = np.load("user_data/user_all/arrays/st2_variance_" + condition_str + "_matrix.npy")

    mean_array_st0 = np.load("user_data/user_all/arrays/st0_mean_" + condition_str + "_matrix.npy")
    mean_array_st1 = np.load("user_data/user_all/arrays/st1_mean_" + condition_str + "_matrix.npy")
    mean_array_st2 = np.load("user_data/user_all/arrays/st2_mean_" + condition_str + "_matrix.npy")
    
    
    
    var_array_save_str_st0 = "user_data/user_all/st0_variance_" + condition_str + "_matrix.png" 
    var_array_save_str_st1 = "user_data/user_all/st1_variance_" + condition_str + "_matrix.png" 
    var_array_save_str_st2 = "user_data/user_all/st2_variance_" + condition_str + "_matrix.png" 

    mean_array_save_str_st0 = "user_data/user_all/st0_mean_" + condition_str + "_matrix.png" 
    mean_array_save_str_st1 = "user_data/user_all/st1_mean_" + condition_str + "_matrix.png" 
    mean_array_save_str_st2 = "user_data/user_all/st2_mean_" + condition_str + "_matrix.png" 


    
    upper_condition_str = condition_str[0].upper() + condition_str[1:]
    var_array_title_str_st0 = f"{upper_condition_str} Variance Matrix - State 0: Stuck" 
    var_array_title_str_st1 = f"{upper_condition_str} Variance Matrix - State 1: Accomplished" 
    var_array_title_str_st2 = f"{upper_condition_str} Variance Matrix - State 2: Progressing" 
    
    mean_array_title_str_st0 = f"{upper_condition_str} Mean Matrix - State 0: Stuck" 
    mean_array_title_str_st1 = f"{upper_condition_str} Mean Matrix - State 1: Accomplished" 
    mean_array_title_str_st2 = f"{upper_condition_str} Mean Matrix - State 2: Progressing" 


    print(f"running plotter @ {step_num}")
    plot_3d_array(var_array_st0, var_array_save_str_st0, var_array_title_str_st0, plotter=None, norm_min=0., norm_max=8., scale_label="variance")
    plot_3d_array(var_array_st1, var_array_save_str_st1, var_array_title_str_st1, plotter=None, norm_min=0., norm_max=8., scale_label="variance")
    plot_3d_array(var_array_st2, var_array_save_str_st2, var_array_title_str_st2, plotter=None, norm_min=0., norm_max=8., scale_label="variance")
    
    plot_3d_array(mean_array_st0, mean_array_save_str_st0, mean_array_title_str_st0, plotter=None)
    plot_3d_array(mean_array_st1, mean_array_save_str_st1, mean_array_title_str_st1, plotter=None)
    plot_3d_array(mean_array_st2, mean_array_save_str_st2, mean_array_title_str_st2, plotter=None)

# User Section State Selections

## Test to see if users had internal consistency between section 2X and 2O

In [ ]:
def get_user_section_state_idxs(user_ID):
    
    # Enter the data in spreadsheet format
    workbook_path = "user_data/response_book.xlsx"
    response_book = load_workbook(workbook_path)
    
    try: # Try to open existing sheet
        response_sheet = response_book.get_sheet_by_name("conv_idx")
    except KeyError:  # If ot doesn't exist. create it
        response_sheet = response_book.create_sheet(title="conv_idx")
        
    
# ROW 1
    response_sheet["A1"] = "user ID"
    response_sheet.merge_cells('A1:A2')

    # STATE 0 - Stuck
    response_sheet["B1"] = "St0 - STUCK - Sect2X"
    response_sheet.merge_cells('B1:D1')
    response_sheet["E1"] = "St0 - STUCK - Sect2O"
    response_sheet.merge_cells('E1:G1')
    response_sheet["H1"] = "St0 - STUCK - Matched"
    response_sheet.merge_cells('H1:J1')

    # STATE 1 - Accomplished
    response_sheet["L1"] = "St1 - ACCOM - Sect2X"
    response_sheet.merge_cells('L1:N1')
    response_sheet["O1"] = "St1 - ACCOM - Sect2O"
    response_sheet.merge_cells('O1:Q1')
    response_sheet["R1"] = "St1 - ACCOM - Matched"
    response_sheet.merge_cells('R1:T1')
    
    # STATE 2 - Progressing
    response_sheet["V1"] = "St2 - PROGR - Sect2X"
    response_sheet.merge_cells('V1:X1')
    response_sheet["Y1"] = "St2 - PROGR - Sect2O"
    response_sheet.merge_cells('Y1:AA1')
    response_sheet["AB1"] = "St2 - PROGR - Matched"
    response_sheet.merge_cells('AB1:AD1')
    
    
# ROW 2
    # STATE 0
    response_sheet["B2"] = "P1"
    response_sheet["C2"] = "P2"
    response_sheet["D2"] = "P3"
    response_sheet["E2"] = "P1"
    response_sheet["F2"] = "P2"
    response_sheet["G2"] = "P3"
    response_sheet["H2"] = "P1"
    response_sheet["I2"] = "P2"
    response_sheet["J2"] = "P3"

    # STATE 1
    response_sheet["L2"] = "P1"
    response_sheet["M2"] = "P2"
    response_sheet["N2"] = "P3"
    response_sheet["O2"] = "P1"
    response_sheet["P2"] = "P2"
    response_sheet["Q2"] = "P3"
    response_sheet["R2"] = "P1"
    response_sheet["S2"] = "P2"
    response_sheet["T2"] = "P3"
    
    # STATE 2
    response_sheet["V2"] = "P1"
    response_sheet["W2"] = "P2"
    response_sheet["X2"] = "P3"
    response_sheet["Y2"] = "P1"
    response_sheet["Z2"] = "P2"
    response_sheet["AA2"] = "P3"
    response_sheet["AB2"] = "P1"
    response_sheet["AC2"] = "P2"
    response_sheet["AD2"] = "P3"
    
    
    user_ID_str = str("{:02d}".format(user_ID))    
    
    sect2X_st0 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2X_final_st0.npy") # Stuck
    sect2X_st1 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2X_final_st1.npy") # Accomplished 
    sect2X_st2 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2X_final_st2.npy") # Progressing
    

    sect2X_st0_max_idx = np.unravel_index(np.argmax(sect2X_st0),sect2X_st0.shape)
    # print(sect2X_st0)
    # print(sect2X_st0_max_idx)
    # print()
    
    sect2X_st1_max_idx = np.unravel_index(np.argmax(sect2X_st1),sect2X_st1.shape)
    # print(sect2X_st1)
    # print(sect2X_st1_max_idx)
    # print()
    
    sect2X_st2_max_idx = np.unravel_index(np.argmax(sect2X_st2),sect2X_st2.shape)
    # print(sect2X_st2)
    # print(sect2X_st2_max_idx)
    # print()
    
    
    sect2O_st0 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2O_final_st0.npy") # Stuck
    sect2O_st1 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2O_final_st1.npy") # Accomplished 
    sect2O_st2 = np.load("user_data/user_" + user_ID_str + "/arrays/" + user_ID_str + "_sect2O_final_st2.npy") # Progressing
    

    sect2O_st0_max_idx = np.unravel_index(np.argmax(sect2O_st0),sect2O_st0.shape)
    # print(sect2O_st0)
    # print(sect2O_st0_max_idx)
    # print()
    
    sect2O_st1_max_idx = np.unravel_index(np.argmax(sect2O_st1),sect2O_st1.shape)
    # print(sect2O_st1)
    # print(sect2O_st1_max_idx)
    # print()
    
    sect2O_st2_max_idx = np.unravel_index(np.argmax(sect2O_st2),sect2O_st2.shape)
    # print(sect2O_st2)
    # print(sect2O_st2_max_idx)
    # print()
    
    
    # Recall    
    # param_1 = Speed of audio loop (BPM)		 --> (100 BPM   //  140 BPM  //  180 BPM)
    # param_2 = Number of beeps per loop (BPL)   --> (1 BPL	 //  2 BPM	 //  4 BPL) 
    # param_3 = Amplitude of Pitch Bend  		 --> (downward  //  neutral  //  upward)
    
    
    
    # Enter the values for tat user
        
    response_sheet["A"+str(user_ID)] = user_ID
    
    response_sheet["B"+str(user_ID)] = sect2X_st0_max_idx[0]
    response_sheet["C"+str(user_ID)] = sect2X_st0_max_idx[1]
    response_sheet["D"+str(user_ID)] = sect2X_st0_max_idx[2]
    response_sheet["E"+str(user_ID)] = sect2O_st0_max_idx[0]
    response_sheet["F"+str(user_ID)] = sect2O_st0_max_idx[1]
    response_sheet["G"+str(user_ID)] = sect2O_st0_max_idx[2]
    # response_sheet["H"+str(user_ID)] = 
    # response_sheet["I"+str(user_ID)] = 
    # response_sheet["J"+str(user_ID)] = 
    
    response_sheet["L"+str(user_ID)] = sect2X_st1_max_idx[0]
    response_sheet["M"+str(user_ID)] = sect2X_st1_max_idx[1]
    response_sheet["N"+str(user_ID)] = sect2X_st1_max_idx[2]
    response_sheet["O"+str(user_ID)] = sect2O_st1_max_idx[0]
    response_sheet["P"+str(user_ID)] = sect2O_st1_max_idx[1]
    response_sheet["Q"+str(user_ID)] = sect2O_st1_max_idx[2]
    # response_sheet["R"+str(user_ID)] = 
    # response_sheet["S"+str(user_ID)] = 
    # response_sheet["T"+str(user_ID)] = 
    
    response_sheet["V"+str(user_ID)] = sect2X_st2_max_idx[0]
    response_sheet["W"+str(user_ID)] = sect2X_st2_max_idx[1]
    response_sheet["X"+str(user_ID)] = sect2X_st2_max_idx[2]
    response_sheet["Y"+str(user_ID)] = sect2O_st2_max_idx[0]
    response_sheet["Z"+str(user_ID)] = sect2O_st2_max_idx[1]
    response_sheet["AA"+str(user_ID)] = sect2O_st2_max_idx[2]
    # response_sheet["AB"+str(user_ID)] = 
    # response_sheet["AC"+str(user_ID)] = 
    # response_sheet["AD"+str(user_ID)] = 
    

    response_book.save(workbook_path)
    
    return sect2X_st0_max_idx, sect2X_st1_max_idx, sect2X_st2_max_idx, sect2O_st0_max_idx, sect2O_st1_max_idx, sect2O_st2_max_idx
    


In [ ]:
param_disc = 3

sect2X_st0_hist_matrix = np.zeros((param_disc, param_disc, param_disc))
sect2X_st1_hist_matrix = np.zeros((param_disc, param_disc, param_disc))
sect2X_st2_hist_matrix = np.zeros((param_disc, param_disc, param_disc))

sect2O_st0_hist_matrix = np.zeros((param_disc, param_disc, param_disc))
sect2O_st1_hist_matrix = np.zeros((param_disc, param_disc, param_disc))
sect2O_st2_hist_matrix = np.zeros((param_disc, param_disc, param_disc))


all_list = [*range(3, 27)]
# condA_list = [3, 4, 5, 6, 7, 8, 9, 10, 22, 23, 24, 25]
# condB_list = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 26] 

for user_num in all_list:
    user_sect2X_st0_max_idx, user_sect2X_st1_max_idx, user_sect2X_st2_max_idx, user_sect2O_st0_max_idx, user_sect2O_st1_max_idx, user_sect2O_st2_max_idx = get_user_section_state_idxs(user_num)
    
    sect2X_st0_hist_matrix[user_sect2X_st0_max_idx[0], user_sect2X_st0_max_idx[1], user_sect2X_st0_max_idx[2]] += 1
    sect2X_st1_hist_matrix[user_sect2X_st1_max_idx[0], user_sect2X_st1_max_idx[1], user_sect2X_st1_max_idx[2]] += 1
    sect2X_st2_hist_matrix[user_sect2X_st2_max_idx[0], user_sect2X_st2_max_idx[1], user_sect2X_st2_max_idx[2]] += 1

    sect2O_st0_hist_matrix[user_sect2O_st0_max_idx[0], user_sect2O_st0_max_idx[1], user_sect2O_st0_max_idx[2]] += 1
    sect2O_st1_hist_matrix[user_sect2O_st1_max_idx[0], user_sect2O_st1_max_idx[1], user_sect2O_st1_max_idx[2]] += 1
    sect2O_st2_hist_matrix[user_sect2O_st2_max_idx[0], user_sect2O_st2_max_idx[1], user_sect2O_st2_max_idx[2]] += 1
    
    
print(f"sect2X_st0_hist_matrix: \n{sect2X_st0_hist_matrix}")
print()
print(f"sect2X_st1_hist_matrix: \n{sect2X_st1_hist_matrix}")
print()
print(f"sect2X_st2_hist_matrix: \n{sect2X_st2_hist_matrix}")
print()
print(f"sect2O_st0_hist_matrix: \n{sect2O_st0_hist_matrix}")
print()
print(f"sect2O_st1_hist_matrix: \n{sect2O_st1_hist_matrix}")
print()
print(f"sect2O_st2_hist_matrix: \n{sect2O_st2_hist_matrix}")

## Now plot each matrix to view distribution

In [ ]:
sect2X_st0_hist_matrix_save = "plots/heatmaps/taskU_st0_conv_heatmap.png"
sect2X_st0_hist_matrix_title = "Subtask U: Uninformed Initialization \nConv Heatmap State 0: Stuck"

sect2X_st1_hist_matrix_save = "plots/heatmaps/taskU_st1_conv_heatmap.png"
sect2X_st1_hist_matrix_title = "Subtask U: Uninformed Initialization \nConv Heatmap State 1: Accomplished"

sect2X_st2_hist_matrix_save = "plots/heatmaps/taskU_st2_conv_heatmap.png"
sect2X_st2_hist_matrix_title = "Subtask U: Uninformed Initialization \nConv Heatmap State 2: Progressing"

sect2O_st0_hist_matrix_save = "plots/heatmaps/taskI_st0_conv_heatmap.png"
sect2O_st0_hist_matrix_title = "Subtask I: Informed Initialization \nConv Heatmap State 0: Stuck"

sect2O_st1_hist_matrix_save = "plots/heatmaps/taskI_st1_conv_heatmap.png"
sect2O_st1_hist_matrix_title = "Subtask I: Informed Initialization \nConv Heatmap State 1: Accomplished"

sect2O_st2_hist_matrix_save = "plots/heatmaps/taskI_st2_conv_heatmap.png"
sect2O_st2_hist_matrix_title = "Subtask I: Informed Initialization \nConv Heatmap State 2: Progressing"

In [ ]:

# sect2X_st0_hist_matrix
plot_3d_array(sect2X_st0_hist_matrix, sect2X_st0_hist_matrix_save, sect2X_st0_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


In [ ]:

# sect2X_st1_hist_matrix
plot_3d_array(sect2X_st1_hist_matrix, sect2X_st1_hist_matrix_save, sect2X_st1_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


In [ ]:

# sect2X_st2_hist_matrix
plot_3d_array(sect2X_st2_hist_matrix, sect2X_st2_hist_matrix_save, sect2X_st2_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


In [ ]:

# sect2O_st0_hist_matrix
plot_3d_array(sect2O_st0_hist_matrix, sect2O_st0_hist_matrix_save, sect2O_st0_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


In [ ]:

# sect2O_st1_hist_matrix
plot_3d_array(sect2O_st1_hist_matrix, sect2O_st1_hist_matrix_save, sect2O_st1_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


In [ ]:

# sect2O_st2_hist_matrix
plot_3d_array(sect2O_st2_hist_matrix, sect2O_st2_hist_matrix_save, sect2O_st2_hist_matrix_title, plotter=None, norm_min=0, norm_max=16, scale_label="Count", colour_map='Greens')


## Also print them as Histograms so you can see quantitative distribution 

In [ ]:
print(f"sect2X_st0_hist_matrix: \n{sect2X_st0_hist_matrix}")
# print()
# print(f"sect2X_st1_hist_matrix: \n{sect2X_st1_hist_matrix}")
# print()
# print(f"sect2X_st2_hist_matrix: \n{sect2X_st2_hist_matrix}")
# print()
# print(f"sect2O_st0_hist_matrix: \n{sect2O_st0_hist_matrix}")
# print()
# print(f"sect2O_st1_hist_matrix: \n{sect2O_st1_hist_matrix}")
# print()
# print(f"sect2O_st2_hist_matrix: \n{sect2O_st2_hist_matrix}")

## Create these plots but as real Histograms

In [ ]:
def hist_matrix_histplotter(hist_matrix, save_str, title_str, plotter=True):

    hist_x_list = []
    hist_y_list = []
    
    for param1_idx_123 in range(0, 3):          # BPM
        for param2_idx_123 in range(0, 3):      # BPL
            for param3_idx_123 in range(0, 3):  # PitchBend
            
                hist_x_list.append(str((param1_idx_123, param2_idx_123, param3_idx_123)))
                hist_y_list.append(hist_matrix[param1_idx_123, param2_idx_123, param3_idx_123])
            
    # print(f"hist_x_list: {len(hist_x_list)} \n{hist_x_list}")
    # print()
    # print(f"hist_y_list: {len(hist_y_list)} \n{hist_y_list}")    
    
    
    fig, axs = plt.subplots(figsize=(10, 6), layout='constrained')
    
    histo = plt.bar(hist_x_list, hist_y_list, color='green', edgecolor='black')

    axs.set_title(title_str, size = 18)
    axs.set_xlabel("\nParam Index [(P1, P2, P3) = (BPM, BPL, Pitch Bend)]\n", size = 15)
    axs.set_ylabel("\nCount\n", size = 15)
    
    axs.grid(color='black', linestyle='-', linewidth=0.1)
    
    axs.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    axs.yaxis.set_ticks(np.arange(0, 18, 1))
    
    fig.autofmt_xdate(rotation=75)

    
    # Save the fig
    plt.savefig(save_str, bbox_inches='tight', pad_inches=0.25)

    
    # Show the fig if plotter is not set to None
    if plotter:
        plt.show()

    if plotter == None:
        print(" Plots closed. ")
        matplotlib.pyplot.close()

In [ ]:
sect2X_st0_hist_2d_save = "plots/histograms/taskU_st0_conv_histogram.png"
sect2X_st0_hist_2d_title = "Subtask U: Uninformed Initialization \nConvergence Histogram State: Stuck\n"

sect2X_st1_hist_2d_save = "plots/histograms/taskU_st1_conv_histogram.png"
sect2X_st1_hist_2d_title = "Subtask U: Uninformed Initialization \nConvergence Histogram State: Accomplished\n"

sect2X_st2_hist_2d_save = "plots/histograms/taskU_st2_conv_histogram.png"
sect2X_st2_hist_2d_title = "Subtask U: Uninformed Initialization \nConvergence Histogram State: Progressing\n"

sect2O_st0_hist_2d_save = "plots/histograms/taskI_st0_conv_histogram.png"
sect2O_st0_hist_2d_title = "Subtask I: Informed Initialization \nConvergence Histogram State: Stuck\n"

sect2O_st1_hist_2d_save = "plots/histograms/taskI_st1_conv_histogram.png"
sect2O_st1_hist_2d_title = "Subtask I: Informed Initialization \nConvergence Histogram State: Accomplished\n"

sect2O_st2_hist_2d_save = "plots/histograms/taskI_st2_conv_histogram.png"
sect2O_st2_hist_2d_title = "Subtask I: Informed Initialization \nConvergence Histogram State 2: Progressing\n"

In [ ]:
hist_matrix_histplotter(sect2X_st0_hist_matrix, save_str=sect2X_st0_hist_2d_save, title_str=sect2X_st0_hist_2d_title)

In [ ]:
hist_matrix_histplotter(sect2X_st1_hist_matrix, save_str=sect2X_st1_hist_2d_save, title_str=sect2X_st1_hist_2d_title)

In [ ]:
hist_matrix_histplotter(sect2X_st2_hist_matrix, save_str=sect2X_st2_hist_2d_save, title_str=sect2X_st2_hist_2d_title)

In [ ]:
hist_matrix_histplotter(sect2O_st0_hist_matrix, save_str=sect2O_st0_hist_2d_save, title_str=sect2O_st0_hist_2d_title)

In [ ]:
hist_matrix_histplotter(sect2O_st1_hist_matrix, save_str=sect2O_st1_hist_2d_save, title_str=sect2O_st1_hist_2d_title)

In [ ]:
hist_matrix_histplotter(sect2O_st2_hist_matrix, save_str=sect2O_st2_hist_2d_save, title_str=sect2O_st2_hist_2d_title)